##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Sovradattamento e sottoadattamento

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/overfit_and_underfit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizza su TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Esegui in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizza il sorgente su GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/it/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Scarica il notebook</a>
  </td>
</table>

Note: La nostra comunità di Tensorflow ha tradotto questi documenti. Poichè queste traduzioni sono *best-effort*, non è garantito che rispecchino in maniera precisa e aggiornata la [documentazione ufficiale in inglese](https://www.tensorflow.org/?hl=en). 
Se avete suggerimenti per migliorare questa traduzione, mandate per favore una pull request al repository Github [tensorflow/docs](https://github.com/tensorflow/docs). 
Per proporsi come volontari alla scrittura o alla review delle traduzioni della comunità contattate la 
[mailing list docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Come sempre, il codice in questo esempio usa le API `tf.keras`, di cui potete imparare di più nella [guida Keras](https://www.tensorflow.org/guide/keras) di TensorFlow.

In entrambi gli esempi precedenti—[classificazione di testo](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub) e [previsione di consumo carburante](https://www.tensorflow.org/tutorials/keras/regression) — abbiamo visto come l'accuratezza del nostro modello sui dati di validazione raggiunga il picco dopo l'addestramento per un certo numero di epoche, e poi ristagni o cominci a decrescere.

In altre parole, come il nostro modello si *sovradatti* (*overfit* n.d.t.) con i dati addestramento. Imparare come gestire il sovradattamento è importante. Sebbene sia spesso possibile raggiungere un'accuratezza alta sul *insieme di addestramento*, ciò che realmente vogliamo è sviluppare un modello che generalizzi bene sul *insieme di test* (o su dati che non ha mai visto prima).

L'opposto del sovradattamento è il *sottoadattamento*. Il sottoadattamento si verifica quando c'è ancora spazio di miglioramento sui dati di test. Ciò può accadere per un certo numero di ragioni: Se il modello non è abbastanza potente, se è sovra-regolarizzato, o, semplicemente, non è stato addestrato abbastanza. Ciò significa che la rete non ha appreso i pattern rilevanti nei dati di addestramento.

Se lo allenate per un tempo troppo lungo, il modello inizierà a sovradattarsi ed ad imparare pattern dai dati di addestramento che non si generalizzeranno ai dati di test. Dobbiamo trovare un compromesso. Capire come addestrare per un numero appropriato di epoche, come vedremo nel seguito, è una competenza utile.

La soluzione migliore per prevenire il sovradattaemnto è usare dati di addestramento più completi. Il dataset dovrebbe coprire lo spettro completo degli input che il modello possa aspettarsi di trattare. Ulteriori dati possono essere utili solo se coprono nuovi ed interessanti casi.

Un modello addestrato su un insieme più completo di dati generalizzerà naturalmente meglio. Quando ciò non fosse possibile, la soluzione migliore è usare tecniche come la regolarizzazione. Esse impongono dei vincoli alla quantità ed al tipo di informazioni che il vostro modello può immagazzinare. Se una rete può memorizzare solo un piccolo numero di pattern, il processo di ottimizzazione lo costringerà a focalizzarsi sui patern più promettenti, che hanno la migliore probabilità di generalizzare bene.

In questo notebook, esploreremo diverse tecniche di regolarizzazione, e le useremo per migliorare un modello di classificazione.

## Setup

Prima di iniziare,importiamo i pacchetti necessari:

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

In [ ]:
!pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile


In [ ]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

## Il dataset Higgs

L'obiettivo di questo tutorial non è fare fisica delle particelle, perciò non approfondiamo i dettagli del dataset. Esso contiene  11&#x202F;000&#x202F;000 esempi, con 28 caratteristiche ciascuno, ed una etichetta di classificazione binaria.

In [ ]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz')

In [ ]:
FEATURES = 28

La classe `tf.data.experimental.CsvDataset` può essere usata per leggere record csv direttamente da un file gzip senza passi intermedi di decompressione.

In [ ]:
ds = tf.data.experimental.CsvDataset(gz,[float(),]*(FEATURES+1), compression_type="GZIP")

La classe che legge il cvs restituisce una lista di scalari per ogni record. La funzione seguente re-impacchetta tale lista in una coppia (vettore_di_caratteristiche, etichetta).

In [ ]:
def pack_row(*row):
  label = row[0]
  features = tf.stack(row[1:],1)
  return features, label

TensorFlow è più efficiente quando tratta grandi lotti di dati.

Così, invece di re-impacchettare ciascuna riga individualmente, costruiamo un nuovo `Dataset` che contiene lotti di 10000-esempi, applichiamo la funzione `pack_row` a ciascun lotto, e quindi separiamo i lotti in singoli record:

In [ ]:
packed_ds = ds.batch(10000).map(pack_row).unbatch()

Diamo uno sguardo ad alcuni dei record di questo nuovo `packed_ds`.

Le caratteristiche non sono perfettamente normalizzate, ma questo non è un problema per questo tutorial.

In [ ]:
for features,label in packed_ds.batch(1000).take(1):
  print(features[0])
  plt.hist(features.numpy().flatten(), bins = 101)

Per mantenere questo tutorial relativamente breve, usiamo solo i primi 1000 campioni per la validazione, ed i successivi 10000 per l'addestramento:

In [ ]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

I metodi `Dataset.skip` e `Dataset.take` rendono semplice la cosa.

Allo stesso tempo, usare il metodo `Dataset.cache` assicura che il caricatore non abbia bisogno di ri-leggerei dati dal file ad ogni nuova epoca:

In [ ]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

In [ ]:
train_ds

Questi dataset restituiscono esempi singoli. Usiamo il metodo `.batch` per creare lotti di dimensione appropriata per l'addestramento. Prima della confezione dei lotti ricordiamo anche di `.shuffle` (*mescolare* n.d.t.) e `.repeat` (*ripetere* n.d.t.) l'insieme di addestramento.

In [ ]:
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

## Dimostrazione di sovradattamento

La via più semplice per prevenire il sovradattamento è cominciare con un modello piccolo: un modello con un piccolo numero di parametri da apprendere (determinati dal numero di livelli e dal numero di unità per livello). Nel deep learning, spesso si fa riferimento al numero di parametri che possono essere appresi da un modello, come la "capacity" del modello.

Intuitivamente, un modello con più parametri avrà maggiore "capacità di memorizzazione" e quindi sarà capace di imparare facilmente una corrispondenza perfetta simil-vocabolario tra i campioni di addestramento ed i relativi obiettivi, una mappatura senza nessun potere di generalizzazione, ma ciò sarebbe inutile per fare previsioni su dati mai visti.

Teniamo sempre a mente: i modelli di deep learning tendono ad essere bravi nell'aderire ai dati di addestramneto, ma la sfida reale è la generalizzazione, non l'adattamento.

d'altra parte, se la rete ha limitate risorse di memorizzazione, non sarà capace di imparare la mappatura così facilmente. Per minimizzare la sua funzione obiettivo, dovrà apprendere rappresentazioni compresse che hanno maggiore potere predittivo. Allo stesso tempo, se fate il vostro modello troppo piccolo, esso ha difficoltà ad aderire ai dati di addestramento. Occorre trovare un punto di equilibrio tra "capacità eccessiva" e "capacità insufficiente".

Sfortunatamente, non c'è nessuna formula magica per stabilire la dimensione giusta o l'architettura del vostro modello (in altre parole, il numero dei livelli, o la dimensione giusta di ciascun livello). Dovrete fare esperimenti usando architetture diverse.

Per trovare la dimensione giusta del modello, è meglio partire con relativamente pochi livelli e parametri, quindi cominciare ad aumentare la dimensione dei livelli o ad aggiungere nuovi livelli fino a che non vedete diminuire gli effetti sulla funzione obiettivo di validazione.

Come punto di partenza, iniziate con un modello semplice che usa solo `layers.Dense`, poi create versioni più grandi e confrontatele.

### Procedura di addestramento

Molti modelli si addestrano meglio se riducete gradualmente il tasso di apprendimento durante l'addestramento. Per ridurre il tasso di apprendimento nel tempo, usiamo `optimizers.schedules`:

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

Il codice di cui sopra usa `schedules.InverseTimeDecay` per diminuire iperbolicamente il tasso di apprendimento ad 1/2 del tasso base dopo 1000 epoche, ad 1/3 a 2000 epoche e così via.

In [ ]:
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')


Ogni modello in questo tutorial userà la medesima configurazione di addestramento. Perciò la impostiamo in modo riusabile, partendo dalla lista delle callback.

L'addestramento di questo tutorial gira per molte epoche brevi. Per ridurre lo sforzo di tracciamento usiamo `tfdocs.EpochDots` che utilizza un semplice `.` per ogni epoca e, un insieme completo di metriche ogni 100 epoche.

In seguito, includiamo `callbacks.EarlyStopping` per evitare lunghi tempi di addestramento non necessari. notate che questa callback è impostata per monitorare la `val_binary_crossentropy`, non la `val_loss`. Questa differenza sarà importante in seguito.

Usiamo `callbacks.TensorBoard` per generare i tracciati TensorBoard per l'addestramento.


In [ ]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

Analogamente, ogni modello userà le stesse impostazioni `Model.compile` e `Model.fit` settings:

In [ ]:
def compile_and_fit(model, name, optimizer=None, max_epochs=10000):
  if optimizer is None:
    optimizer = get_optimizer()
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[
                  tf.keras.losses.BinaryCrossentropy(
                      from_logits=True, name='binary_crossentropy'),
                  'accuracy'])

  model.summary()

  history = model.fit(
    train_ds,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_data=validate_ds,
    callbacks=get_callbacks(name),
    verbose=0)
  return history

### Modello minuscolo

Iniziamo addestrando un modello lineare:

In [ ]:
tiny_model = tf.keras.Sequential([
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(1)
])

In [ ]:
size_histories = {}

In [ ]:
size_histories['Tiny'] = compile_and_fit(tiny_model, 'sizes/Tiny')

Adesso controlliamo cos'ha fatto il modello:

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)
plt.ylim([0.5, 0.7])

### Modello piccolo

Per vedere se possiamo battere le prestazioni del modello piccolo, addestriamo progressivamente modelli più grandi.

Proviamo due livelli nascosti con 16 unità ciascuno:

In [ ]:
small_model = tf.keras.Sequential([
    # `input_shape` is only required here so that `.summary` works.
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(16, activation='elu'),
    layers.Dense(1)
])

In [ ]:
size_histories['Small'] = compile_and_fit(small_model, 'sizes/Small')

### Modello medio

Ora proviamo 3 modelli nascosti con 64 unità ciascuno:

In [ ]:
medium_model = tf.keras.Sequential([
    layers.Dense(64, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(1)
])

Ed addestriamo il modello usando i medesimi dati:

In [ ]:
size_histories['Medium']  = compile_and_fit(medium_model, "sizes/Medium")

### Modello grande

Come esercizio, potete creare un modello ancora più grande, e vedere come comincia a sovradattarsi rapidamente.  Poi, aggiungiamo a questa prova una rete con capacità ancora maggiore, molto più di quanto il problema richiederebbe:

In [ ]:
large_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(1)
])

E ancora, addestrare il modello usando gli stessi dati:

In [ ]:
size_histories['large'] = compile_and_fit(large_model, "sizes/large")

### Traccia delle funzioni obiettivo in addestramento e validazione

Le linee continue mostrano la funzione obiettivo in addestramento, e le linee tratteggiate la funzione obiettivo in validazione (ricordate: una minore funzione obiettivo in validazione indica un modello migliore).

Mentre la costruzione di un modello più grande gli conferisce più potenza, se questa potenza non è vincolata in qualche modo, essa può facilmente sovradattarsi sull'insieme di addestramento.

In questo esempio, tipicamente, solo il modello `"Minuscolo"` riesce ad evitare completamente il sovradattamento, mentre ciascuno dei modelli più grandi si sovradatta ai dati più rapidamente. Ciò diventa così grave oer il modello `"grande"` che siamo costretti a modificare il grafico ad una scala logaritmica per vedere ciò che realmente accade.

La cosa è evidente se si tracciano e confrontano le metriche di validazione e le metriche di addestramento.

* E' normale ci sia una piccola differenza.
* Se entrambe le metriche si muovono nella stessa direzione, va tutto bene.
* Se la metrica di validazione comincia a ristagnare mentre la metrica di addestramento continua a migliorare, siamo probabilmente vicini al sovradattamento.
* Se la metrica di validazione va nella direzione sbagliata, il modello si sta chiaramente sovradattando.

In [ ]:
plotter.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.7])
plt.xlabel("Epochs [Log Scale]")

Notare: tutti gli addestramenti precedenti girano usando `callbacks.EarlyStopping` per fermare l'addestramento una volta che sia chiaro che il modello non stia facendo progressi.

### Visualizzazione in TensorBoard

Tutti questi modelli scrivono tracciati TensorBoard durante l'addestramento.

Per aprire un visualizzatore TensorBoard all'interno di un notebook, copiate il codice seguente in una code-cell:

```
%tensorboard --logdir {logdir}/sizes
```

Possiamo vedere il [risultato di un'esecuzione precedente](https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97) di questo notebook su [TensorBoard.dev](https://tensorboard.dev/).

TensorBoard.dev è un ambiente gestito per ospitare, tracciare e condividere con chiunque esperimenti di ML.

Per semplicità, è anche incluso in un `<iframe>`:

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97",
    width="100%", height="800px")

Se vogliamo condividere risultati TensorBoard, possiamo caricare i tracciati in [TensorBoard.dev](https://tensorboard.dev/) copiando il codice seguente in una code-cell.

Notare: Questo passo richiede un account Google.

```
!tensorboard dev upload --logdir  {logdir}/sizes
```

Attenzione: Questo comando non termina. E' progettato per eseguire l'upload continuo dei risultati di un esperimento di lunga durata. Una volta che i nostri dati sono caricati lo dovete fermare usando l'opzione "interrupt execution (interrompi l'esecuzione n.d.t.)" nel vostro strumento di esecuzione dei notebook.

## Strategie per prevenire il sovradattamento

Prima di andare all'interno del contenuto di questo paragrafo, copiamo i tracciati dell'addestramento del modello `"Minuscolo"` di cui sopra, per usarlo come termine di paragone.

In [ ]:
shutil.rmtree(logdir/'regularizers/Tiny', ignore_errors=True)
shutil.copytree(logdir/'sizes/Tiny', logdir/'regularizers/Tiny')

In [ ]:
regularizer_histories = {}
regularizer_histories['Tiny'] = size_histories['Tiny']

### Aggiunta della regolarizzazione del peso


Potreste avere familiarità con il principio del Rasoio di Occam: date due spiegazioni per qualcosa, la spiegazione che ha più probabilità di essere corretta è la "più semplice", quella che fa meno ipotesi. Ciò si applica anche ai modelli appresi dalle reti neurali: dato un insieme di dati di addestramento ed un'architettura di rete, ci sono molteplici insiemi di valori dei pesi (molteplici modelli) che possono spiegare i dati, ed i modelli più semplici si sovradattano meno facilmente di quelli complessi.

In questo contesto, un "modello semplice" è un modello in cui la distribuzione dei valori dei parametri ha meno entropia (o un modello con meno parametri del tutto, come vediamo del paragrafo sopra). Così, un modo comune di mitigare il sovradattamento è porre dei vincoli alla complessità di una rete forzando i suoi pesi a prendere solo valori piccoli, il che rende la distribuzione dei valori dei pesi "regolare". Ciò è chiamato "regolarizzazione dei pesi", e lo si fa aggiungendo alla funzione dei pesi della rete un costo associato all'avere grandi pesi. Questo costo è di due tipi:

* [Regolarizzazione L1](https://developers.google.com/machine-learning/glossary/#L1_regularization), in cui il costo aggiunto è proporzionale al valore assoluto dei coefficienti dei pesi (cioè a ciò che viene chiamata "norma L1" dei pesi).

* [regolarizzazione L2](https://developers.google.com/machine-learning/glossary/#L2_regularization), in cui il costo aggiunto è proporzionale al quadrato dei coefficienti dei pesi (cioè a quella che è chiamata "norma L2" dei pesi). Nel contesto delle reti neurali, la regolarizzazione L2 è chiamata anche decadimento dei pesi. Non lasciate che il nome vi confonda: il decadimento dei pesi è matematicamente la regolarizzazione L2.

La regolarizzazione L1spinge i pesi esattamente verso lo zero, favorendo un modello sparso. La regolarizzazione L2 penalizzerà i parametri di peso senza renderli sparsi in quanto la penalità tende a zero per pesi piccoli. La ragione per cui L2 è più comune.

In `tf.keras`, la regolarizzazione del peso viene ottenuta passando ai livelli delle istanze di regolarizzazione del peso come argomenti delle parole chiave. Ora andiamo ad aggiungere la regolarizzazione L2.

In [ ]:
l2_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001),
                 input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1)
])

regularizer_histories['l2'] = compile_and_fit(l2_model, "regularizers/l2")

`l2(0.001)` significa che ogni coefficiente nella matrice dei pesi del livello aggiunge `0.001 * weight_coefficient_value**2` al totale **loss** della rete.

Questo è il perché monitoriamo direttamente il `binary_crossentropy`. Perché non ha mescolata in se questa componente di regolarizzazione.

Così, lo stesso modello `"Grande"` con una regolarizzazione di penalità `L2` ha migliori prestazioni:


In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Come potete vedere, il modello `"L2"`-regolarizzato è ora molto più competitivo con il modello `"Minuscolo"`. Questo modello `"L2"` è anche molto più resistente al sovradattamento del modello `"Grande"` su cui si basava pur avendo lo stesso numero di parametri.

#### Maggiori informazioni

Ci sono due cose importanti da notare a proposito di questo tipo di regolarizzazione.

**Primo:** se state scrivendo il vostro proprio ciclo di addestramento, allora dovete essere sicuri di chiedere al modello le sue perdite di regolarizzazione.

In [ ]:
result = l2_model(features)
regularization_loss=tf.add_n(l2_model.losses)

**Secondo:** Questa implementazione funziona aggiungendo le penalizzazioni di peso alla funzione obiettivo del modello, e poi applicando una procedura di ottimizzazione standard.

C'è un secondo approccio che applica invece l'ottimizzatore sulla funzione obiettivo grezza, e poi mentre applica il passo calcolato, l'ottimizzatore applica anche un decadimento di peso. Questo "Decadimento di Peso Disaccoppiato" viene visto in ottimizzatori come `optimizers.FTRL` e `optimizers.AdamW`.

### Aggiunta di un dropout

Il dropout ("punto di rinuncia" n.d.t.) è una delle tecniche di regolarizzazione più efficaci e più comunemente utilizzate per le reti neurali, sviluppata da Hinton ed i suoi studenti all'università di Toronto.

Una spiegazione intuitiva per il dropout è che: in virtù del fatto che i singoli nodi della rete non possono fare affidamento sull'output degli altri, ciascun nodo deve produrre caratteristiche di output utili in se stesse.

Il dropout, applicato ad un livello, consiste nel "dropping out (far decadere n.d.t.)" (cioè impostare a zero) un certo numero di caratteristiche di output del livello, durante l'addestramento. Supponiamo che un dato livello, normalmente, durante l'addestramento, restituisca un vettore [0.2, 0.5, 1.3, 0.8, 1.1] per un dato campione di input; dopo l'applicazione del dropout, questo vettore avrà alcuni valori a zero distribuiti casualmente, es: [0, 0.5,
1.3, 0, 1.1].

Il "tasso di dropout" è la frazione delle caratteristiche azzerate; di solito si fissa tra 0.2 e 0.5. Durante il test, non ci sono unità azzerate, ed invece i valori di uscita del livello sono scalati in basso di un fattore uguale al tasso di dropout, per bilanciare il fatto che ci sono più unità attive che durante l'addestramento.

In `tf.keras` potete inserire il dropout in una rete per mezzo di un livello Dropout, che viene applicato all'output el livello immediatamente precedente.

Aggiungiamo due livelli Dropout nella nostra rete per vedere quanto bene riducano il sovradattamento:

In [ ]:
dropout_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['dropout'] = compile_and_fit(dropout_model, "regularizers/dropout")

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

E' chiaro da questo grafico che entrambi questi approcci di regolarizzazione migliorano il comportamento del modello `"Grande"`. Ma non reggono ancora il confronto con il `"Minuscolo"`.

Nel seguito li proviamo, tutti e due insieme, e vediamo se fanno di meglio.

### Dropout combinati L1 + L2

In [ ]:
combined_model = tf.keras.Sequential([
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['combined'] = compile_and_fit(combined_model, "regularizers/combined")

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Questo modello con la regolarizzazione `"Combinata"` è ovviamente, fin'ora, il migliore.

### Visualizzazione in TensorBoard

Anche questi modelli registrano tracciati TensorBoard.

Per aprire un visualizzatore tensorboard all'interno di un notebook, copiare il codice seguente in una code-cell:

```
%tensorboard --logdir {logdir}/regularizers
```

Potete vedere il [risultato di un'esecuzione precedente](https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97) di questo notebook su [TensorDoard.dev](https://tensorboard.dev/).

Per praticità, esso è anche incluso in un `<iframe>`:

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97",
    width = "100%",
    height="800px")


Che è stato caricato con:

```
!tensorboard dev upload --logdir  {logdir}/regularizers
```

## Conclusioni

Ricapitolando: questi sono i modi più comuni per prevenire il sovradattamento nelle reti neurali:

* Usare più dati di addestramento.
* Ridurre la capacità della rete.
* Aggiungere la regolarizzazione del peso.
* Aggiungere il dropout.

In questa guida non sono trattati due approcci importanti:

* l'aumento dei dati
* la normalizzazione a lotti

Ricordate che ogni metodo è utile in se stesso, ma spesso combinarli può essere più efficace.